In [1]:
import numpy as np
import torch
from torch import nn

# from model.autoEncoder import DenoiseAutoEncoder

#这里很奇怪，直接引用模型会出错。得抄过来......
class DenoiseAutoEncoder(nn.Module):
    def __init__(self):
        super(DenoiseAutoEncoder, self).__init__()
        # Encoder1
        self.Gamma = nn.Sequential(
            nn.Conv2d(3, 10, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(10, 20, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(285180, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
            nn.Sigmoid()
        )

        self.Encoder1 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(3, 64, 3, 1, 1),   # [, 64, 96, 96]
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, 1, 1), # [, 64, 96, 96]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),             # [, 64, 48, 48]
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, 1, 1),  # [, 64, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 3, 1, 1), # [, 128, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, 1, 1), # [, 128, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 256, 3, 1, 1), # [, 256, 48, 48]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                 # [, 256, 24, 24]
            nn.BatchNorm2d(256)   
        )
        
        # decoder1
        self.Decoder1 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3 ,1, 1),   # [, 128, 24, 24]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 128, 3, 2, 1, 1),   # [, 128, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 64, 3, 1, 1),    # [, 64, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(64, 32, 3, 1, 1),      # [, 32, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.ConvTranspose2d(32, 32, 3, 1, 1),      # [, 32, 48, 48]
            nn.ConvTranspose2d(32, 16, 3, 2, 1, 1),  # [, 16, 96, 96]
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.ConvTranspose2d(16, 3, 3, 1, 1),         # [, 3, 96, 96]
            nn.Sigmoid()
        )

         # Encoder2
        self.Encoder2 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(3, 64, 3, 1, 1),   # [, 64, 96, 96]
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, 1, 1), # [, 64, 96, 96]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),             # [, 64, 48, 48]
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, 1, 1),  # [, 64, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 3, 1, 1), # [, 128, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, 1, 1), # [, 128, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 256, 3, 1, 1), # [, 256, 48, 48]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                 # [, 256, 24, 24]
            nn.BatchNorm2d(256)   
        )
        
        # decoder2
        self.Decoder2 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3 ,1, 1),   # [, 128, 24, 24]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 128, 3, 2, 1, 1),   # [, 128, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 64, 3, 1, 1),    # [, 64, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(64, 32, 3, 1, 1),      # [, 32, 48, 48]
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.ConvTranspose2d(32, 32, 3, 1, 1),      # [, 32, 48, 48]
            nn.ConvTranspose2d(32, 16, 3, 2, 1, 1),  # [, 16, 96, 96]
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.ConvTranspose2d(16, 3, 3, 1, 1),         # [, 3, 96, 96]
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoder1 = self.Encoder1(x)
        decoder1 = self.Decoder1(encoder1)
        encoder2 = self.Encoder2(x)
        decoder2 = self.Decoder2(encoder2)
        gamma = self.Gamma(x)

        img_mid = (x*decoder1)+decoder2
        b = img_mid.shape[0]
        img_high = torch.stack([img_mid[i,:,:,:]**gamma[i,:] for i in range(b)], dim=0)

        return img_high
        
DAEmodel = DenoiseAutoEncoder()
DAEmodel.load_state_dict(torch.load('./last_Epoch_3ae.pth',map_location="cpu"))
DAEmodel.eval()


/opt/anaconda3/envs/torchversion_1_12_1/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DenoiseAutoEncoder(
  (Gamma): Sequential(
    (0): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=285180, out_features=50, bias=True)
    (8): ReLU()
    (9): Linear(in_features=50, out_features=1, bias=True)
    (10): Sigmoid()
  )
  (Encoder1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [2]:
#验证模型结果

img = torch.Tensor(1, 3, 400, 600)
print('total parameters:', sum(param.numel() for param in DAEmodel.parameters()))
a = DAEmodel(img)
print(a.shape)

total parameters: 16552055
torch.Size([1, 3, 400, 600])


In [12]:
# 直接用coremltools，pytorch_1.12.1 可用
import torch
import coremltools as ct
import coremltools
random_input = torch.rand(1, 3, 400, 600) 
traced_model = torch.jit.trace(DAEmodel, random_input) 
print("开始转换")
model = ct.convert(
    model=traced_model,
    source="pytorch",
    inputs=[ct.ImageType(name="input_image", shape=random_input.shape, scale=1 / 255.0, color_layout=ct.colorlayout.RGB)],
    # inputs=[ct.TensorType(name="input_tensor",dtype=np.float32, shape=(1,3,400,600))],
    # outputs=[ct.TensorType(name="mid_res"), ct.ImageType(name="output_image",color_layout=ct.colorlayout.RGB)],
    outputs=[ct.TensorType(name="output_tensor",dtype=np.float32)],
    # outputs=["mid_res","output_tensor"],
    minimum_deployment_target=coremltools.target.iOS14, #Currently models that use bilinear upsampling in PyTorch can be converted to CoreML models targeting iOS 14, but not iOS 13.
    convert_to="neuralnetwork",
) 


print("Set feature descriptions (these show up as comments in XCode)")
# model.input_description["input_tensor"] = "a (1,3,96,96) shaped tensor transfered from an image"
# model.output_description["output_tensor"] = "a (1,3,96,96) tensor, *255 and transpose((1,2,0) to convert to an image"
model.author = "Alyosha"
model.license = "leigithub1024/xxx"
model.short_description = "lowlight image enhancement"
model.version = "1.0"
modelName = "model.mlmodel"
model.save(modelName)






# print("将输出结果*255")
# spec = coremltools.utils.load_spec(modelName)
# builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec)
# # builder.add_elementwise(name=f"xx", input_names=[f"mid_res"], output_name=f"mid_res_xx", mode="MULTIPLY", alpha=255)
# builder.add_elementwise(name=f"multiply_xy_by_two_output_image", input_names=[f"output_tensor"], output_name=f"output_image", mode="MULTIPLY", alpha=255)
# builder.set_output(output_names=["output_image"], output_dims=[(3,400,600)])
# # builder.set_output(output_names=["output_image"], output_dims=[(3,96,96)])

# model_spec = builder.spec
# coremltools.models.utils.save_spec(model_spec, modelName)

# print(model_spec.description.output)





开始转换


Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 416/416 [00:14<00:00, 27.90 ops/s]  


Set feature descriptions (these show up as comments in XCode)


In [11]:
# 更改模型输出为图像
import coremltools
import coremltools.proto.FeatureTypes_pb2 as ft 
spec = coremltools.utils.load_spec(modelName)
output = spec.description.output[0]
import coremltools.proto.FeatureTypes_pb2 as ft
output.type.imageType.colorSpace = ft.ImageFeatureType.RGB
output.type.imageType.height = 400
output.type.imageType.width = 600
output.name = 'output_image'
coremltools.utils.save_spec(spec, modelName)
print(spec.description.input,spec.description.output)



[name: "input_image"
type {
  imageType {
    width: 600
    height: 400
    colorSpace: RGB
  }
}
] [name: "output_image"
type {
  imageType {
    width: 600
    height: 400
    colorSpace: RGB
  }
}
]


In [13]:
# 在这个网站验证模型结构：https://netron.app/
# 验证模型能不能跑
# Use PIL to load and resize the image to expected size
from PIL import Image
import matplotlib.pyplot as plt
model = coremltools.models.MLModel(modelName)

#####输入格式：tensor
# example_image = Image.open("1.jpg").resize((96, 96))
# img = cv2.imread('./1.jpg')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img = cv2.resize(img, (96, 96),interpolation = cv2.INTER_LINEAR)
# img = img.astype(np.float32) / 255.0
# # mean = np.array([0.485, 0.456, 0.406])
# # val = np.array([0.229, 0.224, 0.225])
# # img = (img - mean) / val
# print("gemfield debug img shape1: ",img.shape)
# img= img.astype(np.float32)
# img = img.transpose((2,0,1))
# print("gemfield debug img shape2: ",img.shape)
# img = np.expand_dims(img,axis=0)
# print("gemfield debug img shape3: ",img.shape)
# out_dict = model.predict({"input_tensor": img})


##### 输入格式：image
img = Image.open('1.jpg')
img = img.resize((600,400))
# img = img.astype(np.float32) / 255.0
out_dict = model.predict({"input_image": img})



#####输出格式：tensor
# res = out_dict["output_tensor"] 
# arr = np.array((res[0]*255), dtype=np.uint8)
# print("arr shape:", arr.shape)
# arr = arr.transpose((1,2,0))
# print("transposed_arr shape",arr.shape, arr)
# image = Image.fromarray(arr)
# image.show()



##### 输出格式：图像
# print(out_dict)
# output_pil_image = out_dict["output_image"]
# image = output_pil_image.convert("RGB")
# image_array = np.array(image)
# image_array = image_array * 255
# print(image_array)
out_dict['output_image'].show()  #这里发现输出是全黑的，找到问题了，coreml的那个input格式转换没调好

RuntimeError: Error compiling model: "Error reading protobuf spec. validator error: Layer '494' of type 925 has 1 inputs but expects at least 2.".

In [ ]:
##尝试使用 pth => onnx => mlmodel
#转换为onnx
dummy_input = torch.rand(1, 3, 96, 96) #这里高宽可能是颠倒的
input_names = ["gemfield_in"]
output_names = ["gemfield_out"]
torch.onnx.export(DAEmodel,
                  dummy_input,
                  "syszux_scene.onnx",
                  verbose=True,
                  input_names=input_names,
                  output_names=output_names)

In [ ]:
#验证使用onnx推理

import cv2
import onnxruntime
import numpy as np
import sys
import torch

from PIL import Image
from torchvision import transforms

session = onnxruntime.InferenceSession("./syszux_scene.onnx")
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[1].name
input_shape = session.get_inputs()[0].shape
print("gemfield debug required input shape", input_name,output_name,input_shape)

img = cv2.imread('./10.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#INTER_NEAREST, INTER_LINEAR, INTER_AREA, INTER_CUBIC
img = cv2.resize(img, (96, 96),interpolation = cv2.INTER_LINEAR)

img = img.astype(np.float32) / 255.0

mean = np.array([0.485, 0.456, 0.406])
val = np.array([0.229, 0.224, 0.225])
# img = (img - mean) / val
# print(img)

print("gemfield debug img shape1: ",img.shape)
img= img.astype(np.float32)
img = img.transpose((2,0,1))
#img = img.transpose((2,1,0))
print("gemfield debug img shape2: ",img.shape)
img = np.expand_dims(img,axis=0)
print("gemfield debug img shape3: ",img.shape)

res = session.run([output_name], {input_name: img})
arr = np.array((res[0][0]*255), dtype=np.uint8)
print("arr shape:", arr.shape)

arr = arr.transpose((1,2,0))
print("transposed_arr shape",arr.shape, arr)
image = Image.fromarray(arr)
image.save('image.jpg')



In [ ]:
#转换为mlmodel
#报错：No module named 'coremltools.converters.nnssa'，换了好几个环境也无法解决
from onnx_coreml import convert
model = convert(model='syszux_scene.onnx',minimum_ios_deployment_target='13')
model.save('syszux_scene.mlmodel')


In [2]:
import torch

# Create two tensors
a = torch.tensor([[1, 2], [3, 4]])
b = torch.tensor([[2, 0], [1, 2]])

# Compute the dot product of a and b along the last axis of a and the second-to-last axis of b
result = torch.tensordot(a, b, dims=([-1], [-2]))

print(result)  

result = torch.matmul(a,b)
print(result) 



tensor([[ 4,  4],
        [10,  8]])
tensor([[ 4,  4],
        [10,  8]])


In [17]:
import torch
a = torch.randn(1,240000,16)
b = torch.randn(16,16)

result = torch.tensordot(a, b, dims=([-1], [-1]))
print(result, result.shape)  

result1 = torch.matmul(a,b.t())
print(result1, result1.shape) 

tensor([[[ 1.8689, -5.2327, -6.6254,  ..., -3.8065, -1.6950,  4.2786],
         [-1.2535, -5.0202, -2.4007,  ..., -3.3875,  0.0727,  6.7865],
         [ 3.8709,  3.2595, -4.3169,  ..., -4.2282, -0.1526,  3.1204],
         ...,
         [ 5.9847,  7.4674,  3.5788,  ...,  4.4165,  0.4777, -7.7185],
         [ 5.6143,  0.0232, -0.8350,  ...,  6.3332,  0.8166,  1.1386],
         [-2.9651, -1.1399,  3.1023,  ...,  6.5655,  1.4220,  2.8216]]]) torch.Size([1, 240000, 16])
tensor([[[ 1.8689, -5.2327, -6.6254,  ..., -3.8065, -1.6950,  4.2786],
         [-1.2535, -5.0202, -2.4007,  ..., -3.3875,  0.0727,  6.7865],
         [ 3.8709,  3.2595, -4.3169,  ..., -4.2282, -0.1526,  3.1204],
         ...,
         [ 5.9847,  7.4674,  3.5788,  ...,  4.4165,  0.4777, -7.7185],
         [ 5.6143,  0.0232, -0.8350,  ...,  6.3332,  0.8166,  1.1386],
         [-2.9651, -1.1399,  3.1023,  ...,  6.5655,  1.4220,  2.8216]]]) torch.Size([1, 240000, 16])


In [13]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
y = torch.tensor([[1, 2], [3, 4]])
# print(torch.matmul(x,y))
print(torch.tensordot(x, y, dims=([-2], [-2])))

tensor([[13, 18],
        [17, 24],
        [21, 30]])


In [ ]:
#验证coreML推理